In [0]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [0]:
global list_of_dataframes 
list_of_dataframes = []


html_element = "table.outlinetable:nth-child(10)"
page = requests.get("https://cloford.com/resources/codes/index.htm")
soup = BeautifulSoup(page.text, 'html.parser')

tables = soup.find_all('table',class_='outlinetable')
print(f'There were {len(tables)} tables found:\n')

for each_table in tables:
    translate_table_to_dataframe(each_table)

names = ["Country Codes", "Additional Codes", "Additional FIPS 10-4 Codes"]
for one_dataframe, one_name in zip(list_of_dataframes, names):
    one_dataframe.name = one_name
    print(one_dataframe.name)

There were 3 tables found:

Country Codes
Additional Codes
Additional FIPS 10-4 Codes


In [0]:
def translate_table_to_dataframe(table):
    dataframe_friendly_array = []
    dataframe_ready_headers = []
    column_headers = table.find_all('th')
    for header in column_headers:
        dataframe_ready_headers.append(header.text)

    rows = table.find_all('tr')

    for row in rows:
        row_data = []
        for col in row.find_all('td'):
            try:
                row_data.append(col.text)
            except:
                continue
        dataframe_friendly_array.append(row_data)
            
    dataframe = pd.DataFrame(data = dataframe_friendly_array, columns = dataframe_ready_headers)
    list_of_dataframes.append(dataframe)

In [0]:
df_list = [list_of_dataframes[i] for i in range(3)]

final = pd.concat(df_list).drop(index=0)

remove_list = ['-', '--', '\xa0', np.nan]

for column in final.columns:
    
    final.loc[final[column].isin(remove_list), column] = 'No Record'

final.loc[final.Country == 'Namibia', ['ISO (2)', 'Internet']] = 'NA'

final.to_csv('GeoInformation_Country.csv', index = False)

final

,Continent,Region,Country,Capital,FIPS,ISO (2),ISO (3),ISO (No),Internet,Note
1,Asia,South Asia,Afghanistan,Kabul,AF,AF,AFG,004,AF,No Record
2,Europe,South East Europe,Albania,Tirana,AL,AL,ALB,008,AL,No Record
3,Africa,Northern Africa,Algeria,Algiers,AG,DZ,DZA,012,DZ,No Record
4,Oceania,Pacific,American Samoa,Pago Pago,AQ,AS,ASM,016,AS,No Record
5,Europe,South West Europe,Andorra,Andorra la Vella,AN,AD,AND,020,AD,No Record
...,...,...,...,...,...,...,...,...,...,...
22,Asia,South East Asia,Paracel Islands,No Record,PF,No Record,No Record,No Record,No Record,No Record
23,Asia,South East Asia,Spratly Islands,No Record,PG,No Record,No Record,No Record,No Record,No Record
24,Africa,Indian Ocean,Tromelin Island,No Record,TE,No Record,No Record,No Record,No Record,ISO includes with the Miscellaneous (French) I...
25,Oceania,North Pacific Ocean,Wake Island (US),No Record,WQ,No Record,No Record,No Record,No Record,ISO includes with the US Minor Outlying Islands


In [0]:
final.isna().sum()

Continent    0
Region       0
Country      0
Capital      0
FIPS         0
ISO (2)      0
ISO (3)      0
ISO (No)     0
Internet     0
Note         0
dtype: int64